In [3]:
from pymongo import MongoClient
import pprint
import pandas as pd
import requests
from bs4 import BeautifulSoup
import json
import time

Request the URL

In [3]:
url = 'https://www.goodreads.com/list/show/264.Books_That_Everyone_Should_Read_At_Least_Once?page=1'
r = requests.get(url)

In [4]:
r.status_code 

200

Save into MongoDB

In [5]:
client = MongoClient()
db = client.goodreads

In [6]:
page1 = db.must_read_books

In [8]:
page1.insert_one({'html': r.text})

Parse text to get data

In [10]:
soup = BeautifulSoup(r.text, 'html.parser')

This should give me everything that is in the entry for each book - only indexing the first one

In [11]:
soup.find_all('tr', '')[0]

<tr itemscope="" itemtype="http://schema.org/Book">
<td class="number" valign="top">1</td>
<td valign="top" width="5%">
<div class="u-anchorTarget" id="2657"></div>
<div class="js-tooltipTrigger tooltipTrigger" data-resource-id="2657" data-resource-type="Book">
<a href="/book/show/2657.To_Kill_a_Mockingbird" title="To Kill a Mockingbird">
<img alt="To Kill a Mockingbird" class="bookCover" itemprop="image" src="https://i.gr-assets.com/images/S/compressed.photo.goodreads.com/books/1553383690i/2657._SY75_.jpg"/>
</a> </div>
</td>
<td valign="top" width="100%">
<a class="bookTitle" href="/book/show/2657.To_Kill_a_Mockingbird" itemprop="url">
<span aria-level="4" itemprop="name" role="heading">To Kill a Mockingbird</span>
</a> <br>
<span class="by">by</span>
<span itemprop="author" itemscope="" itemtype="http://schema.org/Person">
<div class="authorName__container">
<a class="authorName" href="https://www.goodreads.com/author/show/1825.Harper_Lee" itemprop="url"><span itemprop="name">Harper

Identify the elements that I want

In [19]:
entries = soup.find_all('tr', '')[0]

In [63]:
entries.find('td', 'number').text # ranking
entries.find('a', 'bookTitle').text.strip() # title
entries.find('a', 'bookTitle')['href'] # link to book
entries.find('a', 'authorName').text.rstrip() # author
entries.find('span', 'minirating').text.strip() # rating
entries.find('span', 'smallText uitext').find('a').text # score
entries.find('span', 'smallText uitext').find_all('a')[1].text # votes


'/book/show/2657.To_Kill_a_Mockingbird'

Add front end of url to completed working link

In [66]:
# link_pt1 = 'https://www.goodreads.com'
# link_pt2 = entries.find('a', 'bookTitle')['href']
link_lst = ['https://www.goodreads.com', entries.find('a', 'bookTitle')['href']]
full_link = ''.join(link_lst)

'https://www.goodreads.com/book/show/2657.To_Kill_a_Mockingbird'

In [74]:
b_title = []
author = []
ranking = []
rating = []
score = []
votes = []
links_pt2 = []

In [75]:
for idx, book in enumerate(soup.find_all('tr', '')):
    b_title.append(book.find('a', 'bookTitle').text.strip())
    author.append(book.find('a', 'authorName').text.strip())
    ranking.append(book.find('td', 'number').text)
    rating.append(book.find('span', 'minirating').text.strip())
    score.append(book.find('span', 'smallText uitext').find('a').text)
    votes.append(book.find('span', 'smallText uitext').find_all('a')[1].text)
    links_pt2.append(book.find('a', 'bookTitle')['href'])


In [77]:
b_title

['To Kill a Mockingbird',
 "Harry Potter and the Sorcerer's Stone (Harry Potter, #1)",
 '1984',
 'Pride and Prejudice',
 'The Diary of a Young Girl',
 'Animal Farm',
 'The Little Prince',
 'The Great Gatsby',
 'The Catcher in the Rye',
 'The Lord of the Rings',
 'The Book Thief',
 'Jane Eyre',
 'The Chronicles of Narnia (Chronicles of Narnia, #1-7)',
 'Lord of the Flies',
 'Romeo and Juliet',
 'Harry Potter and the Deathly Hallows (Harry Potter, #7)',
 'The Kite Runner',
 'The Giver (The Giver, #1)',
 'The Giving Tree',
 "Charlotte's Web",
 'Little Women',
 'The Hunger Games (The Hunger Games, #1)',
 'The Hobbit, or There and Back Again',
 'Green Eggs and Ham',
 'Of Mice and Men',
 'Harry Potter and the Prisoner of Azkaban (Harry Potter, #3)',
 'Fahrenheit 451',
 "Alice's Adventures in Wonderland & Through the Looking-Glass",
 'Wuthering Heights',
 'Gone with the Wind',
 'Night  (The Night Trilogy, #1)',
 'The Picture of Dorian Gray',
 'Holy Bible: King James Version',
 'Harry Potter a

Add front end of url to each relative path

In [81]:
def combine_url_parts(lst):
    
    full_links = []
    for idx, link in enumerate(lst):
        link_lst = ['https://www.goodreads.com', lst[idx]]
        full_link = ''.join(link_lst)
        full_links.append(full_link)
    return full_links

In [84]:
# combine_url_parts(links_pt2)
full_links = combine_url_parts(links_pt2)
# full_links

['https://www.goodreads.com/book/show/2657.To_Kill_a_Mockingbird',
 'https://www.goodreads.com/book/show/3.Harry_Potter_and_the_Sorcerer_s_Stone',
 'https://www.goodreads.com/book/show/40961427-1984',
 'https://www.goodreads.com/book/show/1885.Pride_and_Prejudice',
 'https://www.goodreads.com/book/show/48855.The_Diary_of_a_Young_Girl',
 'https://www.goodreads.com/book/show/170448.Animal_Farm',
 'https://www.goodreads.com/book/show/157993.The_Little_Prince',
 'https://www.goodreads.com/book/show/4671.The_Great_Gatsby',
 'https://www.goodreads.com/book/show/5107.The_Catcher_in_the_Rye',
 'https://www.goodreads.com/book/show/33.The_Lord_of_the_Rings',
 'https://www.goodreads.com/book/show/19063.The_Book_Thief',
 'https://www.goodreads.com/book/show/10210.Jane_Eyre',
 'https://www.goodreads.com/book/show/11127.The_Chronicles_of_Narnia',
 'https://www.goodreads.com/book/show/7624.Lord_of_the_Flies',
 'https://www.goodreads.com/book/show/18135.Romeo_and_Juliet',
 'https://www.goodreads.com/b

Go through one sub hyperlinks for more data on book

In [92]:
ex = requests.get(full_links[0])

In [94]:
page1.insert_one({'link': full_links[0], 'html': ex.text})

In [139]:
sub_soup = BeautifulSoup(ex.text, 'html.parser')
sub_soup

<!DOCTYPE html>

<html class="desktop">
<head prefix="og: http://ogp.me/ns# fb: http://ogp.me/ns/fb# good_reads: http://ogp.me/ns/fb/good_reads#">
<title>To Kill a Mockingbird by Harper Lee</title>
<meta content="To Kill a Mockingbird book. Read 91,286 reviews from the world's largest community for readers. The unforgettable novel of a childhood in a sleepy Southe..." name="description"/>
<meta content="telephone=no" name="format-detection"/>
<link href="https://www.goodreads.com/book/show/2657.To_Kill_a_Mockingbird" rel="canonical"/>
<meta content="2415071772" property="fb:app_id"/>
<meta content="books.book" property="og:type"/>
<meta content="To Kill a Mockingbird" property="og:title"/>
<meta content="The unforgettable novel of a childhood in a sleepy Southern town and the crisis of conscience that rocked it, To Kill A Mockingbird becam..." property="og:description"/>
<meta content="https://i.gr-assets.com/images/S/compressed.photo.goodreads.com/books/1553383690i/2657._UY630_SR1200,

Identify elements from subpages

In [183]:
sub_entries = sub_soup.find_all('div', 'row')

In [189]:
sub_entries[0].find('span', {'itemprop': 'bookFormat'}).text # book format
sub_entries[0].find('span', {'itemprop':'numberOfPages'}).text # pages
sub_entries[1].find('nobr', 'greyText').text.strip() # original publication
sub_entries[1].text.strip()

'Published\n        October 1st 2001\n         by HarperCollinsPublishers\n\n            \n              (first published October 15th 1952)'

Go through all sublinks to get data from each one

In [222]:
b_format = []
pg_cnt = []
release = []
new_release = []

In [223]:
for link in full_links:

    sub_page = requests.get(link)
    page1.insert_one({'link': link, 'html': sub_page.text})
    sub_soup = BeautifulSoup(sub_page.text, 'html.parser')

    sub_entries = sub_soup.find_all('div', 'row')
    
    try:
        fmt = sub_entries[0].find('span', {'itemprop': 'bookFormat'}).text # book format
    except:
        fmt = 'NA'
        
    pgs = sub_entries[0].find('span', {'itemprop':'numberOfPages'}).text # pages

    try:
        rls = sub_entries[1].find('nobr', 'greyText').text.strip() # original publication
    except:
        rls = 'NA'

    n_rls = sub_entries[1].text.strip()

    b_format.append(fmt)
    pg_cnt.append(pgs)
    release.append(rls)
    new_release.append(n_rls)

    time.sleep(1)
        


In [196]:
b_format

['Published\n        May 23rd 2006\n         by Harper Perennial Modern Classics \n\n            \n              (first published July 11th 1960)',
 'Published\n        November 1st 2003\n         by Scholastic Inc\n\n            \n              (first published June 26th 1997)',
 'Published\n        September 3rd 2013\n         by Houghton Mifflin Harcourt\n\n            \n              (first published June 8th 1949)',
 'Published\n        October 10th 2000\n         by Modern Library\n\n            \n              (first published January 28th 1813)',
 'Published\n        July 1993\n         by Bantam\n\n            \n              (first published June 25th 1947)',
 'Published\n        April 1996\n         by Signet Classics\n\n            \n              (first published August 17th 1945)',
 'Published\n        June 29th 2000\n         by Harcourt, Inc.\n\n            \n              (first published April 6th 1943)',
 'Published\n        September 2004\n         by Scribner\n\n  

In [227]:
df = pd.DataFrame({'rank': ranking, 'title': b_title, 'author': author, 'format': b_format,
                    'pages': pg_cnt, 'rating': rating, 'score': score,
                    'votes': votes, 'release_date': release, 'new_release_date': new_release})
df

,rank,title,author,format,pages,rating,score,votes,release_date,new_release_date
0,1,To Kill a Mockingbird,Harper Lee,Paperback,324 pages,"4.28 avg rating — 4,499,968 ratings","score: 2,314,644","23,464 people voted",(first published July 11th 1960),Published\n May 23rd 2006\n by ...
1,2,Harry Potter and the Sorcerer's Stone (Harry P...,J.K. Rowling,Hardcover,309 pages,"4.47 avg rating — 7,046,821 ratings","score: 1,776,863","18,118 people voted",(first published June 26th 1997),Published\n November 1st 2003\n ...
2,3,1984,George Orwell,Kindle Edition,237 pages,"4.19 avg rating — 3,139,401 ratings","score: 1,644,982","16,809 people voted",(first published June 8th 1949),Published\n September 3rd 2013\n ...
3,4,Pride and Prejudice,Jane Austen,Paperback,279 pages,"4.26 avg rating — 2,997,377 ratings","score: 1,417,425","14,514 people voted",(first published January 28th 1813),Published\n October 10th 2000\n ...
4,5,The Diary of a Young Girl,Anne Frank,Mass Market Paperback,283 pages,"4.15 avg rating — 2,740,404 ratings","score: 1,349,093","13,817 people voted",(first published June 25th 1947),Published\n July 1993\n by Bant...
...,...,...,...,...,...,...,...,...,...,...
95,96,The Brothers Karamazov,Fyodor Dostoyevsky,Paperback,796 pages,"4.32 avg rating — 243,207 ratings","score: 145,439","1,729 people voted",(first published November 18th 1879),Published\n June 14th 2002\n by...
96,97,The Book of Mormon: Another Testament of Jesus...,Joseph Smith Jr.,Paperback,531 pages,"4.34 avg rating — 75,215 ratings","score: 144,664","1,522 people voted",(first published March 1st 1830),Published\n 2013\n by The Churc...
97,98,The Poisonwood Bible,Barbara Kingsolver,Hardcover,546 pages,"4.06 avg rating — 658,670 ratings","score: 144,042","1,680 people voted",(first published September 24th 1998),Published\n July 5th 2005\n by ...
98,99,"Moby-Dick or, the Whale",Herman Melville,Paperback,654 pages,"3.51 avg rating — 479,050 ratings","score: 140,655","1,790 people voted",(first published October 18th 1851),Published\n February 21st 2003\n ...


In [263]:
url = 'https://www.goodreads.com/list/show/264.Books_That_Everyone_Should_Read_At_Least_Once?page=1'
new_url = url.split('=')
add_1 = int(new_url[1]) + 1
new_url[1] = str(add_1)
'='.join(new_url)
url = new_url

'https://www.goodreads.com/list/show/264.Books_That_Everyone_Should_Read_At_Least_Once?page=2'

In [71]:
b_title = []
author = []
ranking = []
rating = []
score = []
votes = []
b_format = []
pg_cnt = []  
release = []
new_release = []

iter = 0

for i in range(45,49):
    
    url = f'https://www.goodreads.com/list/show/264.Books_That_Everyone_Should_Read_At_Least_Once?page={i}'
    
#     new_url = url.split('=')
#     add_1 = int(new_url[1]) + 1
#     new_url[1] = str(add_1)
#     url = '='.join(new_url)
      
    iter += 1

    r = requests.get(url)

    client = MongoClient()
    db = client.goodreads_clean

    pages = db.must_read_books

    pages.insert_one({'html': r.text})

    soup = BeautifulSoup(r.text, 'html.parser')

    links_pt2 = []

    for idx, book in enumerate(soup.find_all('tr', '')):
        b_title.append(book.find('a', 'bookTitle').text.strip())
        author.append(book.find('a', 'authorName').text.strip())
        ranking.append(book.find('td', 'number').text)
        rating.append(book.find('span', 'minirating').text.strip())
        score.append(book.find('span', 'smallText uitext').find('a').text)
        votes.append(book.find('span', 'smallText uitext').find_all('a')[1].text)
        links_pt2.append(book.find('a', 'bookTitle')['href'])

    def combine_url_parts(lst):

        full_links = []
        for idx, link in enumerate(lst):
            link_lst = ['https://www.goodreads.com', lst[idx]]
            full_link = ''.join(link_lst)
            full_links.append(full_link)
        return full_links

    full_links = combine_url_parts(links_pt2)


    iter_sub = 0

    for link in full_links:
        
        iter_sub += 1
        
        sub_page = requests.get(link)
        
        sub_pages = db.must_read_books_sub
        
        sub_pages.insert_one({'link': link, 'html': sub_page.text})
        
        sub_soup = BeautifulSoup(sub_page.text, 'html.parser')

        sub_entries = sub_soup.find_all('div', 'row')

        try:
            fmt = sub_entries[0].find('span', {'itemprop': 'bookFormat'}).text # book format
        except:
            fmt = 'NA'

        try:
            pgs = sub_entries[0].find('span', {'itemprop':'numberOfPages'}).text # pages
        except:
            pgs = 'NA'
            
        try:
            rls = sub_entries[1].find('nobr', 'greyText').text.strip() # original publication
        except:
            rls = 'NA'

        try:
            n_rls = sub_entries[1].text.strip()
        except:
            n_rls = 'NA'

        b_format.append(fmt)
        pg_cnt.append(pgs)
        release.append(rls)
        new_release.append(n_rls)

        time.sleep(1)
        




In [6]:


client = MongoClient()
db = client.goodreads

pages4 = db.must_read_books

pages4.find_one()


{'_id': ObjectId('5f96f2347a0b3ed6059f52b0'),
 'html': '<!DOCTYPE html>\n<html class="desktop\n">\n<head>\n  <title>Books That Everyone Should Read At Least Once (22949 books)</title>\n\n<meta content="22,619 books based on 99142 votes: To Kill a Mockingbird by Harper Lee, Harry Potter and the Sorcerer\'s Stone by J.K. Rowling, 1984 by George Orwell, Pri..." name=\'description\'>\n<meta content=\'telephone=no\' name=\'format-detection\'>\n<link href=\'https://www.goodreads.com/list/show/264.Books_That_Everyone_Should_Read_At_Least_Once\' rel=\'canonical\'>\n\n\n\n    <script type="text/javascript"> var ue_t0=window.ue_t0||+new Date();\n </script>\n  <script type="text/javascript">\n    var ue_mid = "A1PQBFHBHS6YH1";\n    var ue_sn = "www.goodreads.com";\n    var ue_furl = "fls-na.amazon.com";\n    var ue_sid = "597-4137581-0931708";\n    var ue_id = "0N14BWJ7H3BE9JW4RFAK";\n\n    (function(e){var c=e;var a=c.ue||{};a.main_scope="mainscopecsm";a.q=[];a.t0=c.ue_t0||+new Date();a.d=g;func

In [56]:
iter
iter_sub

100

In [69]:
df = pd.DataFrame({'rank': ranking, 'title': b_title, 'author': author, 'format': b_format,
                    'pages': pg_cnt, 'rating': rating, 'score': score,
                    'votes': votes, 'release_date': release, 'new_release_date': new_release})
df

,rank,title,author,format,pages,rating,score,votes,release_date,new_release_date
0,3994,Complete Works of Oscar Wilde,Oscar Wilde,Hardcover,1270 pages,"4.48 avg rating — 13,921 ratings",score: 269,3 people voted,(first published 1908),Published\n August 4th 2003\n b...
1,3994,Verse for Ages,Colleen Thatcher,Paperback,258 pages,4.72 avg rating — 57 ratings,score: 269,3 people voted,NA,Published\n July 15th 2009\n by...
2,3994,The Unicorn Tree,Cynthia Collins,Paperback,252 pages,4.32 avg rating — 165 ratings,score: 269,3 people voted,(first published January 1st 2012),Published\n May 27th 2018\n by ...
3,3994,"Dragon Prince (Dragon Prince, #1)",Melanie Rawn,Paperback,576 pages,"3.95 avg rating — 15,157 ratings",score: 269,3 people voted,(first published December 1988),Published\n June 7th 2005\n by ...
4,3994,"The Last Town (Wayward Pines, #3)",Blake Crouch,Kindle Edition,294 pages,"4.08 avg rating — 47,264 ratings",score: 269,3 people voted,NA,Published\n July 15th 2014\n by...
...,...,...,...,...,...,...,...,...,...,...
395,4391,رباعيات خيام,Omar Khayyám,Hardcover,184 pages,"4.18 avg rating — 18,249 ratings",score: 231,3 people voted,(first published 1859),Published\n 2007\n \n\n ...
396,4391,Handle with Care,Jodi Picoult,Hardcover,477 pages,"3.98 avg rating — 111,865 ratings",score: 231,3 people voted,(first published 2009),Published\n March 3rd 2009\n by...
397,4398,"Dreaming the Eagle (Boudica, #1)",Manda Scott,Hardcover,465 pages,"4.06 avg rating — 3,677 ratings",score: 230,3 people voted,(first published February 2003),Published\n May 27th 2003\n by ...
398,4398,Dawn of the Dreadfuls (Pride and Prejudice and...,Steve Hockensmith,Paperback,287 pages,"3.48 avg rating — 21,626 ratings",score: 230,3 people voted,(first published March 6th 2007),Published\n March 1st 2010\n by...


In [70]:
df.to_csv('data/book_data_12.csv')

In [42]:
pages.count_documents({})

18

In [43]:
sub_pages.count_documents({})

1700